## Libraries and initialization

In [13]:
# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial
from typing import Tuple, List
from datasets import load_dataset
import tensorflow as tf
from gensim.models import fasttext

In [11]:
dataset = load_dataset("projecte-aina/sts-ca", trust_remote_code=True)

Generating train split: 2073 examples [00:00, 12696.87 examples/s]
Generating validation split: 500 examples [00:00, 34212.40 examples/s]
Generating test split: 500 examples [00:00, 22037.94 examples/s]


In [9]:
WORD_EMBEDDING_FILE = "models/fasttext_100.bin"

In [ ]:

wv_model = fasttext.load_facebook_vectors(WORD_EMBEDDING_FILE)

## Preprocess text

In [17]:
# Define preprocessing
def preprocess(sentence: str) -> List[str]:
    preprocessed = simple_preprocess(sentence)
    return preprocessed

In [12]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["test"].to_list()]

In [15]:
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [simple_preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionari = Dictionary(sentences_pairs_flattened)

In [16]:
# Cálculo de los pesos TF-IDF para las oraciones pre-procesadas
corpus = [diccionari.doc2bow(sent) for sent in sentences_pairs_flattened]
modelo_tfidf = TfidfModel(corpus)

('Atorga per primer cop les mencions Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència Universitària',
 'Creen la menció M. Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència universitària',
 3.5)

In [24]:
corpus = [diccionari.doc2bow(sent) for sent in sentences_pairs_flattened]
modelo_tfidf = TfidfModel(corpus)

In [25]:
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc)
    tf_idf = tf_idf_model[bow]
    vectors, weights = [], []
    for word_index, weight in tf_idf:
        word = dictionary.get(word_index)
        if word in wv_model:
            vectors.append(wv_model[word])
            weights.append(weight)
    return vectors, weights

def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        tf_idf_model: TfidfModel = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess(sentence_1)
        sentence_2_preproc = preprocess(sentence_2)
        # Si usamos TF-IDF
        if tf_idf_model is not None:
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vector1 = np.average(vectors1, weights=weights1, axis=0, )
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        else:
            # Cálculo del promedio de los word embeddings
            vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]
            vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
            vector1 = np.mean(vectors1, axis=0)
            vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [26]:
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped = map_pairs(input_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionari, )
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped_train = map_pairs(input_pairs,  tf_idf_model=modelo_tfidf, dictionary=diccionari, )
mapped_val = map_pairs(input_pairs_val, tf_idf_model=modelo_tfidf, dictionary=diccionari, )
mapped_test = map_pairs(input_pairs_test, tf_idf_model=modelo_tfidf, dictionary=diccionari, )

In [29]:
# Define training constants
batch_size: int = 64
num_epochs: int = 64

In [30]:
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, )

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped_train)
x_val, y_val = pair_list_to_x_y(mapped_val)

In [31]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

## Baseline - cosinus distance

In [ ]:
from scipy.stats import pearsonr
x_test, y_test = pair_list_to_x_y(mapped_test)
# Baseline
def compute_pearson_baseline(x_, y_):
    y_pred_baseline = []
    for v1, v2 in zip(*x_):
        d = 1.0 - spatial.distance.cosine(v1, v2)
        y_pred_baseline.append(d)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred_baseline, y_.flatten())
    return correlation
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (baseline-train): {compute_pearson_baseline(x_train, y_train)}")
print(f"Correlación de Pearson (baseline-validation): {compute_pearson_baseline(x_val, y_val)}")
print(f"Correlación de Pearson (baseline-test): {compute_pearson_baseline(x_test, y_test)}")

## Model profe 1

In [28]:
# Define the Model
import tensorflow as tf

def build_and_compile_model(embedding_size: int = 100, learning_rate: float = 1e-3) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Hidden layer
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 = first_projection(input_1)
    projected_2 = first_projection(input_2)
    
    # Compute the cosine distance using a Lambda layer
    def cosine_distance(x):
        x1, x2 = x
        x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
        x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
        return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))

    output = tf.keras.layers.Lambda(cosine_distance)([projected_1, projected_2])
    # Define output
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adamax(learning_rate))
    return model

In [32]:
# Construir y compilar el modelo
model = build_and_compile_model()
#tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True, )
print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 100)       │     10,100 │ input_layer[0][0… │
│                     │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None)            │          0 │ dense[0][0],      │
│                     │                   │            │ dense[1][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,100 (39.45 KB)

 Trainable params: 10,100 (39.45 KB)

 Non-trainable params: 0 (0.00 B)

None


In [33]:
# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 5.0399 - val_loss: 3.9373
Epoch 2/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.0053 - val_loss: 3.7528
Epoch 3/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.8080 - val_loss: 3.6193
Epoch 4/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.6569 - val_loss: 3.5082
Epoch 5/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.5324 - val_loss: 3.4154
Epoch 6/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.4244 - val_loss: 3.3368
Epoch 7/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3349 - val_loss: 3.2694
Epoch 8/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.2596 - val_loss: 3.2111
Epoch 9/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.1866 - val_loss: 3.1604
Epoch 10/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.1264 - val_loss: 3.1158
Epoch 11/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.0725 - val_loss: 3.0763
Epoch 12/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.0205 - val_l

In [36]:
def compute_pearson(x_, y_):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    y_pred = model.predict(x_)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train)}")
print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val)}")
print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test)}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Correlación de Pearson (train): 0.6363557041608164
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (validation): 0.524443370063694
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (test): 0.5073036010251575


## Model baseline presentació

In [37]:
import tensorflow as tf
def build_and_compile_model(hidden_size: int = 64) -> tf.keras.Model:
  model = tf.keras.Sequential([
      tf.keras.layers.Concatenate(axis=-1, ),
      tf.keras.layers.Dense(hidden_size, activation='relu'),
      tf.keras.layers.Dense(1)
  ])
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model
m = build_and_compile_model()

In [38]:
# Train the model baseline
m.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.5740 - val_loss: 0.7058
Epoch 2/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7131 - val_loss: 0.6637
Epoch 3/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6700 - val_loss: 0.6517
Epoch 4/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6524 - val_loss: 0.6483
Epoch 5/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6423 - val_loss: 0.6439
Epoch 6/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6338 - val_loss: 0.6430
Epoch 7/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6250 - val_loss: 0.6400
Epoch 8/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6186 - val_loss: 0.6383
Epoch 9/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6112 - val_loss: 0.6365
Epoch 10/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6066 - val_loss: 0.6383
Epoch 11/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6011 - val_loss: 0.6404
Epoch 12/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5965 - val_lo

In [39]:
def compute_pearson(x_, y_):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    y_pred = m.predict(x_)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train)}")
print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val)}")
print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test)}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Correlación de Pearson (train): 0.7404950161921382
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (validation): 0.21751544306728543
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (test): 0.34878037980432175


## Model profe better

In [40]:
def build_and_compile_model_better(embedding_size: int = 300, learning_rate: float = 1e-3) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Hidden layer
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 =  first_projection(input_1)
    projected_2 = first_projection(input_2)
    
    # Compute the cosine distance using a Lambda layer
    def normalized_product(x):
        x1, x2 = x
        x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
        x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
        return x1_normalized * x2_normalized

    output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
    output = tf.keras.layers.Dropout(0.1)(output)
    output = tf.keras.layers.Dense(
        16,
        activation="relu",
    )(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(
        1,
        activation="sigmoid",
    )(output)
    
    output = tf.keras.layers.Lambda(lambda x: x * 5)(output)
    
    # Define output
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))
    return model

In [41]:
model_better = build_and_compile_model()

In [42]:
# Train the model baseline
model_better.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.1022 - val_loss: 0.6819
Epoch 2/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6824 - val_loss: 0.6530
Epoch 3/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6595 - val_loss: 0.6471
Epoch 4/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6477 - val_loss: 0.6419
Epoch 5/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6394 - val_loss: 0.6384
Epoch 6/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6309 - val_loss: 0.6358
Epoch 7/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6228 - val_loss: 0.6353
Epoch 8/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6151 - val_loss: 0.6327
Epoch 9/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6068 - val_loss: 0.6322
Epoch 10/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6015 - val_loss: 0.6319
Epoch 11/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5954 - val_loss: 0.6344
Epoch 12/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5887 - val_lo

In [45]:
def compute_pearson(x_, y_):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    y_pred = model_better.predict(x_)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train)}")
print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val)}")
print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test)}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Correlación de Pearson (train): 0.7836179282078864
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (validation): 0.15264317042013342
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (test): 0.2990447546531585


Els models del profe són els del fitxer single vector model

# Del fitxer sequence model 

In [49]:
from typing import Optional

In [47]:
REMAP_EMBEDDINGS: bool = True
USE_PRETRAINED: bool = True
MAX_LEN: int = 96

In [50]:
def map_word_embeddings(
        sentence: str,
        sequence_len: int = MAX_LEN,
        fixed_dictionary: Optional[Dictionary] = None
) -> np.ndarray:
    """
    Map to word-embedding indices
    :param sentence:
    :param sequence_len:
    :param fixed_dictionary:
    :return:
    """
    sentence_preproc = simple_preprocess(sentence)[:sequence_len]
    _vectors = np.zeros(sequence_len, dtype=np.int32)
    index = 0
    for word in sentence_preproc:
        if fixed_dictionary is not None:
            if word in fixed_dictionary.token2id:
                # Sumo 1 porque el valor 0 está reservado a padding
                _vectors[index] = fixed_dictionary.token2id[word] + 1
                index += 1
        else:
            if word in wv_model.key_to_index:
                _vectors[index] = wv_model.key_to_index[word] + 1
                index += 1
    return _vectors


def map_pairs(
    sentence_pairs: List[Tuple[str, str, float]],
    sequence_len: int = MAX_LEN,
    fixed_dictionary: Optional[Dictionary] = None
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        vector1 = map_word_embeddings(sentence_1, sequence_len, fixed_dictionary)
        vector2 = map_word_embeddings(sentence_2, sequence_len, fixed_dictionary)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [52]:
mapped_train = map_pairs(input_pairs, fixed_dictionary=diccionari if REMAP_EMBEDDINGS else None)
mapped_val = map_pairs(input_pairs_val, fixed_dictionary=diccionari if REMAP_EMBEDDINGS else None)
mapped_test = map_pairs(input_pairs_test, fixed_dictionary=diccionari if REMAP_EMBEDDINGS else None)

In [53]:
# MIRAR BÉ PQ FALLA PQ AIXÒ NO SERIA LO MILLOR
import tensorflow as tf
from tensorflow.keras import Layer
class MyLayer(Layer):
    def call(self, x):
        return tf.not_equal(x,0)

In [54]:
# Define model 1

def model_1(
    input_length: int = MAX_LEN,
    dictionary_size: int = 1000,
    embedding_size: int = 16,
    pretrained_weights: Optional[np.ndarray] = None,
    learning_rate: float = 1e-3,
    trainable: bool = False,
    use_cosine: bool = False,
) -> tf.keras.Model:
    # Input layers
    input_1 = tf.keras.Input(shape=(input_length,), dtype=tf.int32)
    input_2 = tf.keras.Input(shape=(input_length,), dtype=tf.int32)

    # Embedding layer
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True
        )
    else:
        dictionary_size = pretrained_weights.shape[0]
        embedding_size = pretrained_weights.shape[1]
        initializer = tf.keras.initializers.Constant(pretrained_weights)
        embedding = tf.keras.layers.Embedding(
            dictionary_size,
            embedding_size,
            input_length=input_length,
            mask_zero=True,
            embeddings_initializer=initializer,
            trainable=trainable,
        )

    # Apply embedding to input sequences
    embedded_1 = embedding(input_1)
    embedded_2 = embedding(input_2)

    # Global average pooling
    _input_mask_1, _input_mask_2 = MyLayer()(input_1), MyLayer()(input_2)
    pooled_1 = tf.keras.layers.GlobalAveragePooling1D()(embedded_1, mask=_input_mask_1)
    pooled_2 = tf.keras.layers.GlobalAveragePooling1D()(embedded_2, mask=_input_mask_2)

    # Compute similarity/distance
    if use_cosine:   
        # Compute the cosine distance using a Lambda layer
        def cosine_distance(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))
        output = tf.keras.layers.Lambda(cosine_distance)([pooled_1, pooled_2])
    else:
        # Compute the cosine distance using a Lambda layer
        def normalized_product(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return x1_normalized * x2_normalized
    
        output = tf.keras.layers.Lambda(normalized_product)([pooled_1, pooled_2])
        output = tf.keras.layers.Dropout(0.1)(output)
        output = tf.keras.layers.Dense(
            16,
            activation="relu",
        )(output)
        output = tf.keras.layers.Dropout(0.2)(output)
        output = tf.keras.layers.Dense(
            1,
            activation="sigmoid",
        )(output)
        
        output = tf.keras.layers.Lambda(lambda x: x * 5)(output)

    # Define the model
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate))

    return model

In [55]:
# Definir constantes de entrenamiento
batch_size: int = 64
num_epochs: int = 128

In [65]:
# Obtener x_train e y_train
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.row_stack(_x_1), np.row_stack(_x_2)), np.array(_y)

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped_train)
x_val, y_val = pair_list_to_x_y(mapped_val)

In [66]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [67]:
_pretrained_weights: Optional[np.ndarray] = None
if USE_PRETRAINED:
    if REMAP_EMBEDDINGS:
        _pretrained_weights = np.zeros(
            (len(diccionari.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in diccionari.token2id.items():
            if token in wv_model:
             _pretrained_weights[_id + 1] = wv_model[token]

            else:
                # In W2V, OOV will not have a representation. We will use 0.
                pass
    else:
        # Not recommended (this will consume A LOT of RAM)
        _pretrained_weights = np.zeros((wv_model.vectors.shape[0] + 1, wv_model.vector_size,),  dtype=np.float32)
        _pretrained_weights[1:, :] = wv_model.vectors

In [68]:
# Build and compile the model
model_nou = model_1(pretrained_weights=_pretrained_weights, trainable=False, use_cosine=False, )
model_nou.summary()

c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_13      │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 96, 100)   │  1,312,600 │ input_layer_12[0… │
│ (Embedding)         │                   │            │ input_layer_13[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_10         │ (None, 96)        │          0 │ input_layer_12[0… │
│ (MyLayer)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_11         │ (None, 96)        │          0 │ input_layer_13[0… │
│ (MyLayer)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 100)       │          0 │ embedding_5[0][0… │
│ (GlobalAveragePool… │                   │            │ my_layer_10[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 100)       │          0 │ embedding_5[1][0… │
│ (GlobalAveragePool… │                   │            │ my_layer_11[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_11 (Lambda)  │ (None, 100)       │          0 │ global_average_p… │
│                     │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 100)       │          0 │ lambda_11[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 16)        │      1,616 │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 16)        │          0 │ dense_15[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 1)         │         17 │ dropout_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_12 (Lambda)  │ (None, 1)         │          0 │ dense_16[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,314,233 (5.01 MB)

 Trainable params: 1,633 (6.38 KB)

 Non-trainable params: 1,312,600 (5.01 MB)

In [69]:
# Train the model
model_nou.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.7322 - val_loss: 0.7246
Epoch 2/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7323 - val_loss: 0.7228
Epoch 3/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7291 - val_loss: 0.7214
Epoch 4/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7289 - val_loss: 0.7195
Epoch 5/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7234 - val_loss: 0.7174
Epoch 6/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7220 - val_loss: 0.7156
Epoch 7/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7167 - val_loss: 0.7133
Epoch 8/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7141 - val_loss: 0.7111
Epoch 9/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7131 - val_loss: 0.7090
Epoch 10/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7078 - val_loss: 0.7070
Epoch 11/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7004 - val_loss: 0.7050
Epoch 12/128
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.

In [70]:
from scipy.stats import pearsonr

x_test, y_test = pair_list_to_x_y(mapped_test)
def compute_pearson(x_, y_, model):
    # Get predictions for the model
    y_pred = model_nou.predict(x_)
    # Compute pearson correlation
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation

# Print results
print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train, model)}")
print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val, model)}")
print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test, model)}")


65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Correlación de Pearson (train): 0.5128442363001666
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Correlación de Pearson (validation): 0.363246755664028
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Correlación de Pearson (test): 0.41946570342474443
